In [1]:
import pandas as pd

In [2]:
def textclean(x):
    x=x.strip()
    x=x.lower()
    x=x[0]
    return x
data=pd.read_csv("cleaned.csv")
if data.columns[0]=='Unnamed: 0':
    data=data.iloc[:,1:]

In [3]:
data.head()

,Self-employed,Number_of_employees_in_org,tech_company,primary_role_in_tech,mental_health_allowance,options_for_mental_health_care,formal_discussion_about_mental_health,Resources_to_seek_help,anonymity_protection,medical_leave_for_depression,...,openly_identified,support_from_coworker,badly_handled,well_handled,tech_industry_support_mental_health,gender,Living_Country,Race,Working_country,Depression
0,0,More than 1000,1.0,0.0,Yes,Yes,Yes,Yes,Yes,Somewhat difficult,...,0,5.0,"Yes, I observed","Yes, I observed",2.0,Female,Canada,White,Canada,Possibly
1,0,More than 1000,1.0,1.0,Yes,Yes,No,I don't know,I don't know,Somewhat difficult,...,0,3.0,No,No,3.0,male,United States of America,White,United States of America,Yes
2,0,6-25,0.0,1.0,Yes,Yes,No,No,I don't know,Somewhat easy,...,0,9.0,No,Maybe/Not sure,3.0,Male,United States of America,White,United States of America,No
3,0,6-25,1.0,1.0,No,No,No,No,I don't know,Neither easy nor difficult,...,0,5.0,No,Maybe/Not sure,1.0,male,Norway,White,Norway,No
4,0,26-100,1.0,1.0,Yes,Yes,Yes,Yes,Yes,Somewhat easy,...,1,5.0,"Yes, I observed","Yes, I observed",4.0,Ostensibly Male,United States of America,White,United States of America,Yes


In [4]:
types=data.dtypes

new_cols=[]
for i in range(len(data.columns)):
    new_cols.append(data.columns[i].strip())

data.columns=new_cols

data.gender=data.gender.apply(textclean)

data.drop(['Living_Country','Race','Working_country'],axis=1,inplace=True)
total_counts={}
#We use counter module for counting
from collections import Counter
#We are using isnan function to check if a vaue is null
from math import isnan
#The below loop builds frequency histogram
for column in data.columns:
    #counts holds frequency values for the current column
    counts=dict(Counter(data[column]))
    #We remove frequency values for nan
    counts={k: counts[k] for k in counts if isinstance(k,str) or not isnan(k)}
    #We sort the frequenct histogram in descending order of frequency
    ordered_counts=sorted(counts.items(),
                          key=lambda x: x[1],
                          reverse=True)
    #We append the sorted values to total_counts 
    total_counts[column]=ordered_counts

labels={}
for i in total_counts.items():
    temp=[]
    for j in range(len(i[1])):
        if i[1][j][1]>int(0.025*len(data)):
            last_label=j
        if i[1][j][1]<int(0.025*len(data)):
            temp.append([i[1][j][0],last_label])
        else:
            temp.append([i[1][j][0],j])
    labels[i[0]]=temp

labels

inverse_labels={}
for i in total_counts.items():
    temp=[]
    for j in range(len(i[1])):
        if i[1][j][1]<int(0.025*len(data)):
            pass
        else:
            temp.append([j,i[1][j][0]])
    inverse_labels[i[0]]=temp

inverse_labels

for column in data.columns:
    for i in labels[column]:
        data[column]=data[column].replace(i[0],i[1])
        print(column,i[0],i[1])

data.head()

for column in data.columns:
    for i in inverse_labels[column]:
        data[column]=data[column].replace(i[0],i[1])
        print(column,i[0],i[1])

data.head()

data.dtypes

In [5]:
data_to_encode=data.select_dtypes(include=['object'])

In [6]:
from sklearn.preprocessing import LabelEncoder

In [15]:
for col in data_to_encode.columns:
    data[col]=LabelEncoder().fit_transform(data[col])

In [16]:
data.head()

,Self-employed,Number_of_employees_in_org,tech_company,primary_role_in_tech,mental_health_allowance,options_for_mental_health_care,formal_discussion_about_mental_health,Resources_to_seek_help,anonymity_protection,medical_leave_for_depression,...,openly_identified,support_from_coworker,badly_handled,well_handled,tech_industry_support_mental_health,gender,Living_Country,Race,Working_country,Depression
0,0,5,1.0,0.0,3,1,2,2,2,3,...,2,5.0,8,8,2.0,19,8,11,4,6
1,0,5,1.0,1.0,3,1,1,0,0,3,...,2,3.0,6,5,3.0,50,46,11,38,7
2,0,4,0.0,1.0,3,1,1,1,0,4,...,2,9.0,6,4,3.0,28,46,11,38,5
3,0,4,1.0,1.0,1,0,1,1,0,2,...,2,5.0,6,4,1.0,50,32,11,26,5
4,0,2,1.0,1.0,3,1,2,2,2,4,...,3,5.0,8,8,4.0,37,46,11,38,7


In [17]:
data.corr()

,Self-employed,Number_of_employees_in_org,tech_company,primary_role_in_tech,mental_health_allowance,options_for_mental_health_care,formal_discussion_about_mental_health,Resources_to_seek_help,anonymity_protection,medical_leave_for_depression,...,openly_identified,support_from_coworker,badly_handled,well_handled,tech_industry_support_mental_health,gender,Living_Country,Race,Working_country,Depression
Self-employed,1.000000,0.400845,0.213931,0.076080,0.273778,0.310766,-0.128412,-0.022294,-0.274191,0.228895,...,-0.061951,-0.094928,-0.013055,-0.025227,-0.097679,-0.069568,-0.176527,0.031895,-0.162921,-0.039993
Number_of_employees_in_org,0.400845,1.000000,-0.053433,0.104394,0.136953,0.170621,-0.019802,0.061704,-0.079743,0.009743,...,-0.086145,-0.159572,0.032828,-0.056427,-0.018132,-0.064687,-0.059551,-0.028875,-0.030451,0.030601
tech_company,0.213931,-0.053433,1.000000,0.017892,0.064385,0.124033,-0.003933,-0.008948,-0.093403,0.195565,...,-0.037651,0.120092,0.040834,0.164243,0.013573,-0.002368,-0.034171,0.014676,-0.013023,0.017415
primary_role_in_tech,0.076080,0.104394,0.017892,1.000000,-0.040953,-0.046486,-0.128892,-0.114228,-0.035453,0.030679,...,-0.126877,0.017387,0.089466,0.002698,0.001266,0.095176,0.077232,-0.014298,0.060501,-0.026895
mental_health_allowance,0.273778,0.136953,0.064385,-0.040953,1.000000,0.559353,0.129538,0.267227,0.173923,0.217561,...,0.115171,0.158191,0.035692,0.163424,0.195293,-0.165648,0.119332,0.010231,0.097005,0.184324
options_for_mental_health_care,0.310766,0.170621,0.124033,-0.046486,0.559353,1.000000,0.147039,0.298005,0.153436,0.131397,...,0.101758,0.056738,-0.002121,0.080176,0.110544,-0.036952,0.001382,-0.001453,-0.017766,0.222601
formal_discussion_about_mental_health,-0.128412,-0.019802,-0.003933,-0.128892,0.129538,0.147039,1.000000,0.508647,0.205210,0.167446,...,0.191058,0.132209,0.003369,0.138051,0.157755,-0.076081,0.007295,0.078254,-0.004079,0.069793
Resources_to_seek_help,-0.022294,0.061704,-0.008948,-0.114228,0.267227,0.298005,0.508647,1.000000,0.268148,0.076014,...,0.189370,0.046592,0.063235,0.136412,0.058880,-0.017376,0.038949,0.014776,0.050883,0.117407
anonymity_protection,-0.274191,-0.079743,-0.093403,-0.035453,0.173923,0.153436,0.205210,0.268148,1.000000,0.202947,...,0.049186,0.138727,0.046041,0.113159,0.220457,0.103996,0.092122,-0.047185,0.097071,0.009113
medical_leave_for_depression,0.228895,0.009743,0.195565,0.030679,0.217561,0.131397,0.167446,0.076014,0.202947,1.000000,...,0.060322,0.317881,-0.010866,0.192839,0.264262,0.038419,-0.038304,0.039753,-0.040224,-0.077822


In [18]:
y=data['Depression']

In [19]:
y

0      6
1      7
2      5
3      5
4      7
5      7
6      7
7      7
8      6
9      4
10     7
11     7
12     5
13     6
14     6
15     7
16     7
17     7
18     4
19     7
20     7
21     7
22     5
23     6
24     5
25     5
26     5
27     6
28     7
29     0
      ..
387    7
388    4
389    7
390    7
391    5
392    7
393    5
394    4
395    7
396    7
397    7
398    6
399    5
400    5
401    7
402    6
403    7
404    7
405    5
406    7
407    6
408    6
409    7
410    5
411    7
412    5
413    7
414    7
415    6
416    7
Name: Depression, Length: 417, dtype: int64

In [21]:
x=data.drop('Depression',axis=1)

In [22]:
x

,Self-employed,Number_of_employees_in_org,tech_company,primary_role_in_tech,mental_health_allowance,options_for_mental_health_care,formal_discussion_about_mental_health,Resources_to_seek_help,anonymity_protection,medical_leave_for_depression,...,employer_in_interview,openly_identified,support_from_coworker,badly_handled,well_handled,tech_industry_support_mental_health,gender,Living_Country,Race,Working_country
0,0,5,1.0,0.0,3,1,2,2,2,3,...,5,2,5.0,8,8,2.0,19,8,11,4
1,0,5,1.0,1.0,3,1,1,0,0,3,...,5,2,3.0,6,5,3.0,50,46,11,38
2,0,4,0.0,1.0,3,1,1,1,0,4,...,5,2,9.0,6,4,3.0,28,46,11,38
3,0,4,1.0,1.0,1,0,1,1,0,2,...,5,2,5.0,6,4,1.0,50,32,11,26
4,0,2,1.0,1.0,3,1,2,2,2,4,...,8,3,5.0,8,8,4.0,37,46,11,38
5,0,1,1.0,1.0,3,1,1,0,0,4,...,2,4,5.0,1,5,3.0,50,46,11,38
6,0,5,1.0,1.0,3,1,1,1,0,1,...,5,2,5.0,6,5,2.0,9,13,11,8
7,0,2,1.0,1.0,2,1,1,1,0,1,...,5,2,8.0,6,7,4.0,50,33,11,27
8,0,5,1.0,1.0,0,0,1,1,1,1,...,5,2,0.0,5,5,1.0,51,35,11,29
9,0,5,0.0,1.0,3,0,1,2,0,3,...,5,2,1.0,8,5,1.0,28,46,11,38


In [23]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y)

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [27]:
model=RandomForestClassifier().fit(xtrain,ytrain)

/home/paree/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [28]:
pred=model.predict(xtest)

In [30]:
from sklearn.metrics import accuracy_score

In [31]:
accuracy_score(ytest,pred)

0.6476190476190476